In [ ]:
import pandas as pd
from os import walk

In [ ]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
datasets_names

In [ ]:
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print(name,'adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print(name,'adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
print('total rows:', len(dataFrame))
dataFrame.head()

# Categorizando Columnas

In [ ]:
text_cols = ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about"];
dummyable_cols = ["instant_bookable","is_business_travel_ready","cancellation_policy",
                  "require_guest_phone_verification","calculated_host_listings_count",
                  "calculated_host_listings_count_entire_homes","review_scores_communication",
                  "review_scores_location","review_scores_value","requires_license",
                  "require_guest_profile_picture","calculated_host_listings_count_private_rooms",
                  "host_location","host_response_time",
                 "calculated_host_listings_count_shared_rooms","host_is_superhost","host_neighbourhood",
                 "host_listings_count","host_has_profile_pic","host_identity_verified",
                 "has_availability","number_of_reviews","neighbourhood_cleansed","city","state","market",
                  "country_code","property_type","room_type","accommodates","bathrooms",
                 "bedrooms","beds","bed_type","guests_included"];
dropeable_cols = ["id","listing_url","scrape_id","last_scraped","thumbnail_url","medium_url",
                  "picture_url","xl_picture_url","host_url","host_name","host_id","host_thumbnail_url",
                 "host_picture_url","host_total_listings_count","calendar_updated","calendar_last_scraped",
                  "neighbourhood","neighbourhood_group_cleansed","zipcode","smart_location","country",
                 "experiences_offered"];
continuos_cols = ["reviews_per_month","host_response_rate","host_acceptance_rate",
                  "number_of_reviews_ltm","review_scores_rating","review_scores_cleanliness",
                  "review_scores_checkin","availability_30","availability_60","availability_90","availability_365",
                 "review_scores_accuracy","minimum_nights","maximum_nights","minimum_minimum_nights",
                  "maximum_minimum_nights","minimum_maximum_nights","maximum_maximum_nights",
                  "minimum_nights_avg_ntm","maximum_nights_avg_ntm"]
date_cols = ["host_since","first_review","last_review"]
explotable_cols = ["street","host_verifications","jurisdiction_names","amenities"]
geo_cols = ["latitude","longitude","is_location_exact"]

target_col = ["price","weekly_price","monthly_price","security_deposit","cleaning_fee","extra_people"]
#license ?

In [ ]:
#dataFrame.drop(text_cols,axis=1,inplace=True);
#dataFrame.drop(dummyable_cols,axis=1,inplace=True);
#dataFrame.drop(dropeable_cols,axis=1,inplace=True);
#dataFrame.drop(continuos_cols,axis=1,inplace=True);
#dataFrame.drop(date_cols,axis=1,inplace=True);
#dataFrame.drop(explotable_cols,axis=1,inplace=True);
#dataFrame.drop(geo_cols,axis=1,inplace=True);

#dataFrame.drop(target_col,axis=1,inplace=True);

# Ejemplos de las columnas categorizadas

In [ ]:
#Columnas detectadas como texto
dataFrame.sample(5).loc[:,text_cols].T

In [ ]:
#Columnas detectadas como candidatas para hacer Dummies
dataFrame.sample(5).loc[:,dummyable_cols].T

In [ ]:
#Columnas a ser dropeadas directamente
dataFrame.sample(5).loc[:,dropeable_cols].T

In [ ]:
#Columnas detectadas como valores continuos, para reescalar
dataFrame.sample(5).loc[:,continuos_cols].T

In [ ]:
#Columnas detectadas como fecha 
dataFrame.sample(5).loc[:,date_cols].T

In [ ]:
#Columnas que habria que trabajar para separarles los datos
dataFrame.sample(5).loc[:,explotable_cols].T

In [ ]:
#Columnas detectadas como informacion de geolocalizacion
dataFrame.sample(5).loc[:,geo_cols].T

In [ ]:
#Columnas detectadas como que pueden filtrar informacion del target
dataFrame.sample(5).loc[:,target_col].T

# Test de modelado solo con Dummies

In [ ]:
# drop outliers
dataFrame["price"]=dataFrame["price"].str.replace('$', '').str.replace(',', '').astype(float)

Q1 = dataFrame["price"].quantile(0.02)
Q3 = dataFrame["price"].quantile(0.98)

dataFrame.drop(dataFrame[dataFrame["price"] > Q3].index,axis=0,inplace=True);
dataFrame.drop(dataFrame[dataFrame["price"] < Q1].index,axis=0,inplace=True);

dataFrame["price"].describe()

In [ ]:
import numpy as np

# generamos las dummies y limpiamos el vector target
X = pd.get_dummies(dataFrame[dummyable_cols].astype(str), drop_first=True).reset_index(drop=True)
y = dataFrame["price"].reset_index(drop=True);
X.shape

In [ ]:
# separamos entrenamiento y test
from sklearn.model_selection import train_test_split
train_data, val_data, train_targets, val_targets = train_test_split(X, y, test_size=0.2, random_state=42)

train_data.shape

In [ ]:
train_targets

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(train_data.shape[1],),kernel_regularizer=regularizers.l1(0.001)))
model.add(Dense(50, activation='relu',kernel_regularizer=regularizers.l1(0.001)))
model.add(Dense(50, activation='relu',kernel_regularizer=regularizers.l1(0.001)))
model.add(Dense(1,kernel_regularizer=regularizers.l1(0.001)))
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mae'])
model.fit(train_data, train_targets,epochs=5, batch_size=500, verbose=1)

In [ ]:
model.evaluate(val_data, val_targets, verbose=0)